In [36]:
import os
import pickle
import pandas as pd
import shutil
import json

In [37]:
downloaded_items_path = '/Users/kofmanya/Desktop/HSE/My assignments/Diploma/Dataset/downloaded_items.pkl'
items_path = '/Users/kofmanya/Desktop/HSE/My assignments/Diploma/Dataset/items.pkl'
images_path = '/Users/kofmanya/Desktop/HSE/My assignments/Diploma/Dataset/images'
dest_path = '/Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/images'

with open(downloaded_items_path, 'rb') as file:
    saved_items = pickle.load(file)

with open(items_path, 'rb') as file:
    items = pickle.load(file)

In [38]:
for item in saved_items.keys():
    print(f"Downloaded: {len(saved_items[item])} {item}")

Downloaded: 1000 damenbekleidung-kleider
Downloaded: 1000 damenbekleidung-shirts
Downloaded: 1000 damenbekleidung-jeans
Downloaded: 1000 damenbekleidung-jacken
Downloaded: 1000 damenbekleidung-jacken-maentel
Downloaded: 1000 damenbekleidung-blusen-tuniken


In [39]:
first_key = next(iter(saved_items.keys()))
print(saved_items[first_key][0])

saint-tropez-freizeitkleid-night-sky-s2821c0op-k11


In [ ]:
first_key = next(iter(items.keys()))
print(items[first_key])

In [41]:
#classes = ['dresses', 'tshirts_and_tops', 'jeans', 'jackets_and_blazers', 'coats', 'shirts_and_blouses']

classes_dict = {
    'damenbekleidung-kleider': 'dresses',
    'damenbekleidung-shirts': 'tshirts_and_tops',
    'damenbekleidung-jeans': 'jeans',
    'damenbekleidung-jacken': 'jackets_and_blazers',
    'damenbekleidung-jacken-maentel': 'coats',
    'damenbekleidung-blusen-tuniken': 'shirts_and_blouses'
}

In [55]:
import os
import shutil

MAX_FILES_PER_CATEGORY = 1020

for item in items.values():
    if item['id'] in saved_items[item['category']]:
        category = classes_dict[item['category']]
        dest = f"{dest_path}/{category}"

        os.makedirs(dest, exist_ok=True)

        files = {f for f in os.listdir(dest) if os.path.isfile(os.path.join(dest, f))}  
        current_file_count = len(files)

        if current_file_count >= MAX_FILES_PER_CATEGORY:
            continue

        for image in item['images']:
            filename = image.split('/')[-1]
            image_name = image.replace('https://img01.ztat.net/article/', '')
            
            if image_name not in files:
                src = f"{images_path}/{image_name}"
                dest_file_path = f"{dest}/{filename}"

                if os.path.isfile(src):
                    src_size = os.path.getsize(src)
                    if src_size == 0:
                        continue
                        
                    try:
                        shutil.copy(src, dest)
                        dest_size = os.path.getsize(dest_file_path)
    
                        if src_size != dest_size:
                            print(f"File size mismatch: {src} -> {dest_file_path}")
                        else:
                            files.add(image_name)
                            current_file_count += 1
                    except Exception as e:
                        print(f"Error copying {src} to {dest}: {e}")
                        os.remove(dest_file_path)
            if current_file_count >= MAX_FILES_PER_CATEGORY:
                break

MAX_FILES_PER_CATEGORY = 1000

Error copying /Users/kofmanya/Desktop/HSE/My assignments/Diploma/Dataset/images/spp-media-p1/e171a82209c8410ab4a9c3e67e4ebd35/60f40c9649504eee9c31e47416c3df28.jpg to /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/images/jackets_and_blazers: [Errno 60] Operation timed out: '/Users/kofmanya/Desktop/HSE/My assignments/Diploma/Dataset/images/spp-media-p1/e171a82209c8410ab4a9c3e67e4ebd35/60f40c9649504eee9c31e47416c3df28.jpg' -> '/Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/images/jackets_and_blazers/60f40c9649504eee9c31e47416c3df28.jpg'


In [63]:
import os

def remove_ds_store(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file == '.DS_Store':
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Error removing {file_path}: {e}")

# Specify the root directory where you want to remove .DS_Store files
root_directory = "/Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/"
remove_ds_store(root_directory)

Removed: /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/.DS_Store
Removed: /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/data/.DS_Store


In [57]:
for dir in os.listdir(dest_path):
    full_path = os.path.join(dest_path, dir)
    if os.path.isdir(full_path):
        files = [f for f in os.listdir(full_path)]
        for file in files[MAX_FILES_PER_CATEGORY:]:
            file_path = os.path.join(full_path, file)
            try:
                os.remove(file_path)
            except Exception as e:
                print(f'Error deleting {file}: {e}')

In [64]:
for dir in os.listdir(dest_path):
    full_path = os.path.join(dest_path, dir)
    if os.path.isdir(full_path):
        files = [f for f in os.listdir(full_path)]
        print(len(files))

1000
1000
1000
1000
1000
1000


In [59]:
meta_path = "/Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/meta"

os.makedirs(meta_path, exist_ok=True)

train_txt = os.path.join(meta_path, "train.txt")
test_txt = os.path.join(meta_path, "test.txt")
train_json = os.path.join(meta_path, "train.json")
test_json = os.path.join(meta_path, "test.json")

train_data = {}
test_data = {}

for dir in os.listdir(dest_path):
    full_path = os.path.join(dest_path, dir)
    if os.path.isdir(full_path):
        train_data[dir] = []
        test_data[dir] = []

print(train_data)
print(test_data)

{'coats': [], 'jackets_and_blazers': [], 'jeans': [], 'dresses': [], 'shirts_and_blouses': [], 'tshirts_and_tops': []}
{'coats': [], 'jackets_and_blazers': [], 'jeans': [], 'dresses': [], 'shirts_and_blouses': [], 'tshirts_and_tops': []}


In [60]:
classes_txt = os.path.join(meta_path, "classes.txt")
labels_txt = os.path.join(meta_path, "labels.txt")

classes = ['coats', 'dresses', 'jackets_and_blazers', 'jeans', 'shirts_and_blouses', 'tshirts_and_tops']
labels = ['Coats', 'Dresses', 'Jackets_and_blazers', 'Jeans', 'Shirts_and_blouses', 'Tshirts_and_tops']

with open(classes_txt, 'w') as f:
    for class_name in classes:
        f.write(class_name + '\n')

with open(labels_txt, 'w') as f:
    for label in labels:
        f.write(label + '\n')

print(f"Classes written to {classes_txt}")
print(f"Labels written to {labels_txt}")

Classes written to /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/meta/classes.txt
Labels written to /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/meta/labels.txt


In [61]:
SEPARATE_INDEX = int(MAX_FILES_PER_CATEGORY * 0.75)

with open(train_txt, 'w') as train_file, open(test_txt, 'w') as test_file:
    for dir in os.listdir(dest_path):
        full_path = os.path.join(dest_path, dir)
        if os.path.isdir(full_path):
            files = [f for f in os.listdir(full_path)]
            for file in files[:SEPARATE_INDEX]:
                path = os.path.join(dir, file)
                train_file.write(path + "\n")
                train_data[dir].append(path)
            for file in files[SEPARATE_INDEX:]:
                path = os.path.join(dir, file)
                test_file.write(path + "\n")
                test_data[dir].append(path)

with open(train_json, 'w') as train_json_file:
    json.dump(train_data, train_json_file, indent=4)

with open(test_json, 'w') as test_json_file:
    json.dump(test_data, test_json_file, indent=4)

In [62]:
with open(train_txt, 'r') as file:
    line_count_train = sum(1 for line in file)

print(f"The file {train_txt} contains {line_count_train} lines.")

with open(test_txt, 'r') as file:
    line_count_test = sum(1 for line in file)

print(f"The file {test_txt} contains {line_count_test} lines.")

The file /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/meta/train.txt contains 4500 lines.
The file /Users/kofmanya/Desktop/HSE/My assignments/year 2/LMSL2/Final_SGA/zalando/meta/test.txt contains 1500 lines.
